# 개요

## 서론
재원이는 축구를 좋아한다. 평소에도 FIFA 게임을 즐기고 있으며, 최근 개최된 2018 러시아 월드컵에도 큰 관심을 가지고 관람했다.  
뿐만 아니라 해외축구까지도 섭렵하며 유러피안 리그도 그의 큰 관심사 중 하나이다.  
평소 축구는 좋아하지만 베팅에는 큰 관심이 없던 재원이도 이번 월드컵에서 배당률이 5000 이상 뛰는 걸 보고 호기심이 생겼다.  
베팅 사이트를 들어가보니 각 경기마다 배당률이 매겨지는데, 많은 축구 팬들이 베팅을 했기 때문인지 배당률만 봐도 어느정도 판세가 보임을 알 수 있었다.  
그래서 굳이 베팅을 하지는 않더라도, 미리 경기의 결과를 예측함에 있어서 의미있는 요인이라는 생각이 들었다.
하지만 단순히 배당률만 보고 그대로 따라간다면 예측의 의미가 있을까?  
그래서 문득 데이터 분석을 배운 김에 승부 예측을 향상시킬 방법을 찾아보기로 결심했다!  

## 분석 대상
1. 업체신뢰도: 베팅업체 배당률과 실제 예측성공률을 비교
2. 배당률 패턴(승:무:패 / 비슷함, 승쏠림, 패쏠림, 무쏠림)에 따라서 실제 결과가 어떻게 나오는지 분석해보기
3. 팀별 상성
4. 매치데이터를 통한 팀 성향 분류 및 상성 분석

## 결론
기본적으로 베팅업체의 배당률만 봐도 대략적으로 경기 결과(승/무/패)를 예측할 수 있다.
따라서 (홈팀을 기준으로) 승(W), 무(D), 패(L)의 확률을 계산해볼 수 있다.

>P(W) = a
P(D) = b
P(L) = c
; a+b+c = 1

다만 배당률만 보고 가기엔 너무나 도박적이기 때문에 *배당률 패턴, 팀별 상성, 팀 성향 상성* 등의 요인을 추가하여 확률을 보정하고싶다.
식을 보정하는 방법은 나중에 생각해보자.

# 업체신뢰도
일단 1x2 betting odds를 기준으로 분석을 해보자.  
그리고 업체 신뢰도를 판단하기에 앞서 업체의 수가 너무 많은 관계로 필터링을 거쳐보자.


In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

df1718 = pd.read_csv("http://www.football-data.co.uk/mmz4281/1718/E0.csv", sep=',', header = 0)
df1617 = pd.read_csv("http://www.football-data.co.uk/mmz4281/1617/E0.csv", sep=',', header = 0)
df1516 = pd.read_csv("http://www.football-data.co.uk/mmz4281/1516/E0.csv", sep=',', header = 0)
df = pd.concat([df1718,df1617,df1516], ignore_index = True)


다음과 같이 B365, BW 두 업체의 배당률을 통해 비교해보자.

In [2]:
B365_odds = ['B365H', 'B365D', 'B365A']
BW_odds = ['BWH', 'BWD', 'BWA']
df[['FTR']+B365_odds + BW_odds].head()

,FTR,B365H,B365D,B365A,BWH,BWD,BWA
0,H,1.53,4.5,6.50,1.50,4.60,6.75
1,A,11.00,5.5,1.33,11.00,5.25,1.30
2,A,1.25,6.5,15.00,1.22,6.50,12.50
3,A,1.83,3.6,5.00,1.80,3.50,4.75
4,H,1.70,3.8,5.75,1.70,3.60,5.50


예상대로, 5경기뿐이지만 두 업체모두 배당률은 비슷한 패턴을 보인다.
또한 실제로 배당(수치)이 제일 작은 결과가 실제 결과와 일치하는 것을 볼 수 있다.  
그렇다면 먼저 간단하게 다음을 살펴보자.
> 업체 예측 결과가 실제 결과가 같을 확률  
; 업체 예측 결과 = 배당률이 가장 낮은 결과


In [3]:
# 값이 최소인 라벨을 구해서 그 라벨의 맨 끝자리[-1]를 보면 H/D/A로 예측 값을 알수있고, 그걸 실제 결과인 FTR 과 비교하자!!!

bets = ['B365', 'IW', 'LB', 'PS', 'WH', 'VC']    # BW 는 918 행에 NaN이 들어가서 뺐다.
results = ['H', 'D', 'A']

for bet in bets:
    odds = [bet+result for result in results]
    guess = df[odds].idxmin(axis=1).map(lambda x: x[-1])
    df[bet] = (df['FTR']==guess)
    
bets_rate = df[bets].apply(pd.value_counts)
bets_rate.loc['Hit rate'] = bets_rate.loc[True] / bets_rate.sum()
bets_rate

,B365,IW,LB,PS,WH,VC
True,622.000000,624.000000,623.000000,621.000000,619.000000,621.000000
False,518.000000,516.000000,517.000000,519.000000,521.000000,519.000000
Hit rate,0.545614,0.547368,0.546491,0.544737,0.542982,0.544737


미미한 차이가 있기는 하지만
> 전체적으로 배당률을 보고 단순히 승/무/패 결과를 예측하는 것은...
 *배당업체에 따른 차이가 없어보인다.*
 
평균적으로 54.5% 정도의 예측 성공률을 나타내고 있는데, 이 정도면 실제 결과 예측과 충분한 관계성이 보인다.
자칫 경기 결과를 승/패 2가지 경우로 오해할 수 있는데, 어디까지나 이 베팅은 승/무/패 3가지 경우를 예측한 것이다.
따라서 아무 기반도 없다면 33%에 그쳐버리는 예측 성공률을 54.5%까지는 올릴 수 있었다.  

하지만 아직 이 정도 수치로는 부족하다. 의미있는 예측을 하기 위해 다른 관점에서 분석해보도록 하자.

단순히 승/무/패 예측이 성공했냐로 끝나지 않고,
> 승/무/패 배당률에 비례해서 배당업체가 얼마나 강하게 예측했고, 그 성공률을 분석해보자.

